# Importing important libraries
Ce notebook contient des exemples de code en Python avec des commentaires. Vous pouvez utiliser des cellules Markdown pour expliquer les concepts en détail et structurer votre notebook.

## Sous-titre

Voici un exemple de fonction simple en Python :


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math

In [2]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("C:/Users/cestm/Downloads/statistics/ranking.csv")

# Extract specific columns 'TEAM_ID', 'STANDINGSDATE', 'TEAM' and filter for 'CONFERENCE'=='West'
selected_columns = df[['TEAM_ID', 'STANDINGSDATE', 'TEAM', 'CONFERENCE']][df['CONFERENCE'] == 'West']

# Filter rows where 'STANDINGSDATE' ends with "-12-22"
filtered_rows = selected_columns[selected_columns['STANDINGSDATE'].str.endswith('-04-02')]

# Display the filtered DataFrame
print(filtered_rows)




           TEAM_ID STANDINGSDATE          TEAM CONFERENCE
3960    1610612756    2022-04-02       Phoenix       West
3961    1610612763    2022-04-02       Memphis       West
3962    1610612744    2022-04-02  Golden State       West
3963    1610612742    2022-04-02        Dallas       West
3964    1610612743    2022-04-02        Denver       West
...            ...           ...           ...        ...
118537  1610612762    2015-04-02          Utah       West
118538  1610612743    2015-04-02        Denver       West
118539  1610612758    2015-04-02    Sacramento       West
118540  1610612747    2015-04-02   L.A. Lakers       West
118541  1610612750    2015-04-02     Minnesota       West

[284 rows x 4 columns]


In [3]:
rankingyear2022 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2022-')]
rankingyear2021 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2021-')]
rankingyear2020 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2020-')]
rankingyear2019 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2019-')]
rankingyear2018 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2018-')]
rankingyear2017 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2017-')]
rankingyear2016 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2016-')]
rankingyear2015 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2015-')]
rankingyear2014 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2014-')]

# Assuming you have already defined rankingyear2022, rankingyear2021, ..., rankingyear2014

# Concatenate the DataFrames along the rows
dff = pd.concat([rankingyear2022, rankingyear2021, rankingyear2020, rankingyear2019, rankingyear2018, rankingyear2017, rankingyear2016, rankingyear2015, rankingyear2014])

# Display the concatenated DataFrame
print(dff)


          TEAM_ID STANDINGSDATE          TEAM CONFERENCE
3960   1610612756    2022-04-02       Phoenix       West
3961   1610612763    2022-04-02       Memphis       West
3962   1610612744    2022-04-02  Golden State       West
3963   1610612742    2022-04-02        Dallas       West
3964   1610612743    2022-04-02        Denver       West
...           ...           ...           ...        ...
35784  1610612743    2014-04-02        Denver       West
35785  1610612740    2014-04-02   New Orleans       West
35786  1610612758    2014-04-02    Sacramento       West
35787  1610612747    2014-04-02   L.A. Lakers       West
35788  1610612762    2014-04-02          Utah       West

[135 rows x 4 columns]


In [4]:

dff.to_csv("CLASSEMENT")


In [20]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("CLASSEMENTT.csv")

# Define initial Elo ratings for each team
initial_elo = 1500  # Initial Elo rating
elo_ratings = {team: initial_elo for team in df['TEAM'].unique()}

# Define Elo rating update function based on rank group difference
def update_elo(rank_diff):
    K = 500  # Increased Elo rating update factor for even stronger variance
    expected_win = 1 / (1 + 10 ** (rank_diff / 400))
    return K * expected_win

# Iterate over rows to update Elo ratings
for index, row in df.iterrows():
    team = row['TEAM']
    rank = (index % 15) + 1  # Calculate the rank dynamically based on the row index
    rank_group_diff = rank - 1  # Calculate the rank group difference
    elo_diff = update_elo(rank_group_diff)
    elo_ratings[team] += elo_diff

# Create a DataFrame for Elo ratings
elo_df = pd.DataFrame(elo_ratings.items(), columns=['TEAM', 'ELO'])
elo_df.drop(elo_df.tail(1).index, inplace=True)
sorted_df = elo_df.sort_values(by='ELO', ascending=False)

# Display the DataFrame
print(sorted_df)


             TEAM          ELO
10   Golden State  3729.859835
0          Denver  3719.064402
14        Houston  3711.875526
1         Memphis  3711.153207
11  Oklahoma City  3706.117068
13    San Antonio  3704.681483
8          Dallas  3702.522862
7        Portland  3702.519682
12    L.A. Lakers  3700.368043
6            Utah  3699.645116
3         Phoenix  3693.175320
2     New Orleans  3688.858891
5      Sacramento  3688.854985
9       Minnesota  3686.700772
4     LA Clippers  3478.415255


In [22]:
import math

def match_outcome_probability(team1, team2):
    # Access Elo ratings using .loc[] for label-based indexing
    elo_team1 = elo_df.loc[elo_df['TEAM'] == str(team1), 'ELO'].values[0]
    elo_team2 = elo_df.loc[elo_df['TEAM'] == str(team2), 'ELO'].values[0]

    delta_elo = elo_team1 - elo_team2
    probability_team1_win = 1 / (1 + math.pow(10, -delta_elo / 400))
    probability_team2_win = 1 / (1 + math.pow(10, delta_elo / 400))
    #probability_draw = 1 - probability_team1_win - probability_team2_win
    return print(f"{probability_team1_win, probability_team2_win}")

print(match_outcome_probability('Golden State', 'Minnesota'))


(0.5617933658926907, 0.43820663410730937)
None


In [10]:


# Read the CSV file into a DataFrame
dfa = pd.read_csv("C:/Users/cestm/Downloads/statistics/ranking.csv")

# Extract specific columns 'TEAM_ID', 'STANDINGSDATE', 'TEAM' and filter for 'CONFERENCE'=='West'
selected_columns = dfa[['TEAM_ID', 'STANDINGSDATE', 'TEAM', 'CONFERENCE']][df['CONFERENCE'] == 'East']

# Filter rows where 'STANDINGSDATE' ends with "-12-22"
filtered_rowss = selected_columns[selected_columns['STANDINGSDATE'].str.endswith('-04-02')]

# Display the filtered DataFrame
print(filtered_rowss)


           TEAM_ID STANDINGSDATE          TEAM CONFERENCE
8235    1610612748    2022-04-02         Miami       East
8236    1610612749    2022-04-02     Milwaukee       East
8237    1610612738    2022-04-02        Boston       East
8238    1610612755    2022-04-02  Philadelphia       East
8239    1610612761    2022-04-02       Toronto       East
...            ...           ...           ...        ...
207142  1610612754    2015-04-02       Indiana       East
207143  1610612765    2015-04-02       Detroit       East
207144  1610612753    2015-04-02       Orlando       East
207145  1610612755    2015-04-02  Philadelphia       East
207146  1610612752    2015-04-02      New York       East

[285 rows x 4 columns]


In [12]:
rankingyear2022 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2022-')]
rankingyear2021 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2021-')]
rankingyear2020 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2020-')]
rankingyear2019 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2019-')]
rankingyear2018 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2018-')]
rankingyear2017 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2017-')]
rankingyear2016 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2016-')]
rankingyear2015 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2015-')]
rankingyear2014 = filtered_rowss[filtered_rowss['STANDINGSDATE'].str.startswith('2014-')]

# Assuming you have already defined rankingyear2022, rankingyear2021, ..., rankingyear2014

# Concatenate the DataFrames along the rows
dffa = pd.concat([rankingyear2022, rankingyear2021, rankingyear2020, rankingyear2019, rankingyear2018, rankingyear2017, rankingyear2016, rankingyear2015, rankingyear2014])

# Display the concatenated DataFrame
print(dffa)

           TEAM_ID STANDINGSDATE          TEAM CONFERENCE
8235    1610612748    2022-04-02         Miami       East
8236    1610612749    2022-04-02     Milwaukee       East
8237    1610612738    2022-04-02        Boston       East
8238    1610612755    2022-04-02  Philadelphia       East
8239    1610612761    2022-04-02       Toronto       East
...            ...           ...           ...        ...
124012  1610612765    2014-04-02       Detroit       East
124013  1610612738    2014-04-02        Boston       East
124014  1610612753    2014-04-02       Orlando       East
124015  1610612755    2014-04-02  Philadelphia       East
124016  1610612749    2014-04-02     Milwaukee       East

[135 rows x 4 columns]


In [13]:

dffa.to_csv("CLASSEMENTS")

In [15]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("CLASSEMENTS.csv")

# Define initial Elo ratings for each team
initial_elo = 1500  # Initial Elo rating
elo_ratings = {team: initial_elo for team in df['TEAM'].unique()}

# Define Elo rating update function based on rank group difference
def update_elo(rank_diff):
    K = 500  # Increased Elo rating update factor for even stronger variance
    expected_win = 1 / (1 + 10 ** (rank_diff / 400))
    return K * expected_win

# Iterate over rows to update Elo ratings
for index, row in df.iterrows():
    team = row['TEAM']
    rank = (index % 15) + 1  # Calculate the rank dynamically based on the row index
    rank_group_diff = rank - 1  # Calculate the rank group difference
    elo_diff = update_elo(rank_group_diff)
    elo_ratings[team] += elo_diff

# Create a DataFrame for Elo ratings
elo_df = pd.DataFrame(elo_ratings.items(), columns=['TEAM', 'ELO'])
elo_df.drop(elo_df.tail(1).index, inplace=True)
sorted_df = elo_df.sort_values(by='ELO', ascending=False)

# Display the DataFrame
print(sorted_df)

            TEAM            ELO
4        Toronto  135420.797796
2         Boston  134945.993189
0          Miami  134686.935949
1      Milwaukee  134644.051364
12       Indiana  134039.603082
6      Cleveland  133996.847157
7        Atlanta  133694.599402
5        Chicago  133694.398600
10    Washington  133694.309961
9      Charlotte  133694.275665
3   Philadelphia  133651.629755
8       Brooklyn  133521.971055
13       Detroit  132658.854398
11      New York  132313.709930
